In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.9188


In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# First Convolutional Layer
x_image = tf.reshape(x, [-1, 28, 28, 1])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# FC layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# FC layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# train
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_cross_entropy = cross_entropy.eval(feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0               
            })
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0
            })
            print('step %d, training accuracy %g, loss %g' % (i, train_accuracy, train_cross_entropy))
        train_step.run(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 0.5
        })
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images,
        y_: mnist.test.labels,
        keep_prob: 1.0
    }))

step 0, training accuracy 0.06, loss 5.47956
step 100, training accuracy 0.16, loss 2.167
step 200, training accuracy 0.3, loss 1.77042
step 300, training accuracy 0.4, loss 1.47447
step 400, training accuracy 0.78, loss 0.834109
step 500, training accuracy 0.9, loss 0.393845
step 600, training accuracy 0.98, loss 0.144464
step 700, training accuracy 0.94, loss 0.243398
step 800, training accuracy 0.94, loss 0.204163
step 900, training accuracy 0.96, loss 0.0876476
step 1000, training accuracy 0.92, loss 0.373548
step 1100, training accuracy 0.96, loss 0.104698
step 1200, training accuracy 1, loss 0.0354371
step 1300, training accuracy 1, loss 0.018142
step 1400, training accuracy 0.98, loss 0.0724009
step 1500, training accuracy 1, loss 0.052944
step 1600, training accuracy 0.96, loss 0.0769887
step 1700, training accuracy 0.98, loss 0.097741
step 1800, training accuracy 1, loss 0.0703579
step 1900, training accuracy 0.98, loss 0.0995655
step 2000, training accuracy 0.96, loss 0.35761

step 16800, training accuracy 1, loss 0.00992863
step 16900, training accuracy 1, loss 0.000185698
step 17000, training accuracy 1, loss 0.00187206
step 17100, training accuracy 1, loss 0.00170306
step 17200, training accuracy 1, loss 0.000804634
step 17300, training accuracy 1, loss 0.000184506
step 17400, training accuracy 1, loss 0.000389433
step 17500, training accuracy 1, loss 0.0015652
step 17600, training accuracy 1, loss 0.000816651
step 17700, training accuracy 0.98, loss 0.0869583
step 17800, training accuracy 1, loss 0.000414459
step 17900, training accuracy 1, loss 0.000439983
step 18000, training accuracy 1, loss 0.000152509
step 18100, training accuracy 1, loss 0.000365074
step 18200, training accuracy 1, loss 0.000890149
step 18300, training accuracy 1, loss 0.000110281
step 18400, training accuracy 1, loss 0.000368375
step 18500, training accuracy 1, loss 0.000455803
step 18600, training accuracy 1, loss 0.0103813
step 18700, training accuracy 1, loss 0.000204773
step 1

In [1]:
with tf.Session() as sess:
    '''
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_cross_entropy = cross_entropy.eval(feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0               
            })
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0
            })
            print('step %d, training accuracy %g, loss %g' % (i, train_accuracy, train_cross_entropy))
        train_step.run(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 0.5
        })
    '''
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images,
        y_: mnist.test.labels,
        keep_prob: 1.0
    }))

NameError: name 'tf' is not defined